In [ ]:
def extract_article_with_selenium(url: str, wait_time: int = 5) -> dict:
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        driver.get(url)
        time.sleep(wait_time)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        title = soup.title.string if soup.title else None

        article_tags = soup.find_all(["article", "section", "div"], limit=5)
        article_text = ""

        for tag in article_tags:
            paragraphs = tag.find_all("p")
            if len(paragraphs) > 3:
                article_text = "\n".join(p.get_text(strip=True) for p in paragraphs)
                break

        if not article_text:
            article_text = "\n".join(p.get_text(strip=True) for p in soup.find_all("p"))

        return {
            "url": url,
            "title": title,
            "content": article_text  # 🔥 sem limite aqui!
        }

    finally:
        driver.quit()

# 3. Teste com Bloomberg ou InfoMoney
test_url = "https://www.infomoney.com.br/economia/haddad-confirma-aliquota-de-175-sobre-aplicacoes-financeiras-e-aumento-para-jcp/"  # Substitua por URL válida
result = extract_article_with_selenium(test_url)
print("Título:", result["title"])
print("Trecho do artigo:\n", result["content"])


Título: Haddad confirma alíquota de 17,5% sobre aplicações financeiras e aumento para JCP
Trecho do artigo:
 Reuters
10/06/2025 14h11•Atualizado 4 horas atrás
Publicidade
O ministro da Fazenda, Fernando Haddad, confirmou nesta terça-feira que o governo vai propor uma alíquota unificada de 17,5% de Imposto de Renda sobre rendimentos de aplicações financeiras, e disse que também será proposto aumentar tributação de Juros sobre Capital Próprio (JCP) de 15% para 20%.
Em entrevista a jornalistas após reunião com o presidente Luiz Inácio Lula da Silva, o ministro disse que enviou as medidas à Casa Civil nesta terça, e que espera que elas cheguem à mesa do presidente ainda nesta data.
“Não mexe com o dia a dia da população, eu considerei as medidas muito mais estruturais e justas do ponto de vista tributário. É uma agenda que interessa à Fazenda, fazer justiça tributária”, disse.
Haddad falou ao retornar ao ministério do Palácio da Alvorada, explicando que conversou com Lula o mesmo que trato

In [2]:
import urllib.robotparser
from urllib.parse import urlparse
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def is_allowed_by_robots(url: str, user_agent: str = "*") -> bool:
    """Verifica se a URL é permitida segundo o robots.txt do domínio."""
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    robots_url = f"{base_url}/robots.txt"

    rp = urllib.robotparser.RobotFileParser()
    try:
        rp.set_url(robots_url)
        rp.read()
        return rp.can_fetch(user_agent, url)
    except Exception as e:
        print(f"Erro ao acessar robots.txt de {base_url}: {e}")
        return False  # Por padrão, nega se falhar

# URL de exemplo
url = "https://www1.folha.uol.com.br/mercado/2025/06/abra-seu-coracao-para-acordo-com-o-mercosul-diz-lula-a-macron.shtml"
user_agent = "Mozilla/5.0"

if is_allowed_by_robots(url, user_agent=user_agent):
    # Configura o Selenium
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(f"user-agent={user_agent}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    driver.quit()

    # Processa com BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
    text_parts = [p.text for p in soup.find_all("p")]
    full_text = "\n".join(text_parts)

    print(full_text[:1000], "...")
else:
    print("⚠️ Acesso à URL bloqueado por robots.txt")


Gostaria de receber as principais notícias do Brasil e do mundo?
Gostaria de receber as principais notícias do Brasil e do mundo?

      Acesse seus artigos salvos em 
Minha Folha,  sua área personalizada
    

      Acesse os artigos do assunto seguido na 
Minha Folha, sua área personalizada
    

benefício do assinante
Você tem 7 acessos por dia para dar de presente. Qualquer pessoa que não é assinante poderá ler.
benefício do assinante
Assinantes podem liberar 7 acessos por dia para conteúdos da Folha.

              Recurso exclusivo para assinantes
            


                assine
              

                ou
              

                faça login
              

Gostaria de receber as principais notícias do Brasil e do mundo?
benefício do assinante
Você tem 7 acessos por dia para dar de presente. Qualquer pessoa que não é assinante poderá ler.
benefício do assinante
Assinantes podem liberar 7 acessos por dia para conteúdos da Folha.

              Recurso exclusivo

In [1]:
# %%capture
# Instalação das bibliotecas necessárias

import requests
from PyPDF2 import PdfReader
import pdfplumber
from io import BytesIO

# Função para extrair texto do PDF (usando duas bibliotecas para maior confiabilidade)
def extract_pdf_text(url):
    try:
        # Download do PDF
        response = requests.get(url)
        response.raise_for_status()  # Verifica erros na requisição
        
        # Método 1: PyPDF2 (mais rápido)
        with BytesIO(response.content) as data:
            reader = PdfReader(data)
            text_pypdf = ""
            for page in reader.pages:
                text_pypdf += page.extract_text() + "\n\n"
        
        # Método 2: pdfplumber (melhor para formatação complexa)
        with pdfplumber.open(BytesIO(response.content)) as pdf:
            text_plumber = ""
            for page in pdf.pages:
                text_plumber += page.extract_text() + "\n\n"
        
        return {
            "PyPDF2_text": text_pypdf.strip(),
            "pdfplumber_text": text_plumber.strip()
        }
    
    except Exception as e:
        return f"Erro: {str(e)}"

# Exemplo de uso
url_cvm = "https://www.rad.cvm.gov.br/ENET/frmDownloadDocumento.aspx?Tela=ext&descTipo=IPE&CodigoInstituicao=1&numProtocolo=1200495&numSequencia=725221&numVersao=1"
resultado = extract_pdf_text(url_cvm)

# Mostrando os resultados
print("=== Texto extraído com PyPDF2 (primeiros 500 caracteres) ===")
print(resultado["PyPDF2_text"][:500] + "\n...")

print("\n=== Texto extraído com pdfplumber (primeiros 500 caracteres) ===")
print(resultado["pdfplumber_text"][:500] + "\n...")

=== Texto extraído com PyPDF2 (primeiros 500 caracteres) ===
www.petrobras.com.br/ri  
Para mais informações : 
PETRÓLEO BRASILEIRO S.A. – PETROBRAS | Relações com Investidores  
 e-mail: petroinvest@petrobras.com.br/acionistas@petrobras.com.br  
 Av. Henrique Valadares, 28 – 19 Andar –  20231 -030 – Rio de Janeiro, RJ.  
 Tel.: 55 (21) 3224- 1510/9947 | 0800- 282 -1540 
 
Este documento pode conter previsões segundo o significado da Seção 27A da Lei de Valores Mobiliários de 1933, conforme alter ada (Lei de Valores Mobiliários) e Seção 
21E da lei de Neg
...

=== Texto extraído com pdfplumber (primeiros 500 caracteres) ===
Petrobras esclarece notícias divulgadas na mídia
—
Rio de Janeiro, 28 de fevereiro de 2024 – A Petróleo Brasileiro S.A. – Petrobras, em relação
às notícias divulgadas na mídia, informa que não há qualquer decisão tomada em relação à
distribuição de dividendos ainda não declarados.
As decisões da Administração sobre dividendos, incluindo a proposta de destinação do
r